# Installing dependencies

In [1]:
# we use the latest version of transformers, peft, and accelerate
!pip install -q accelerate peft transformers

# install bitsandbytes for quantization
!pip install -q bitsandbytes

# install trl for the SFT library
!pip install -q trl

# we need sentencepiece for the llama2 slow tokenizer
!pip install sentencepiece

# we need einops, used by falcon-7b, llama-2 etc
# einops (einsteinops) is used to simplify tensorops by making them readable
!pip install -q -U einops

# we need to install datasets for our training dataset
!pip install -q datasets


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


# Loading model Mistral 7b and dataset

In [1]:
# The model that we want to train from the Hugging Face hub
model_name = "mistralai/Mistral-7B-Instruct-v0.2"

# The instruction dataset to use found on HuggingFace
dataset_name = "KonradSzafer/stackoverflow_python_preprocessed"

# Fine-tuned model name
new_model = "Mistral-7B-Stackoverflow"
output_dir = "./results"

In [2]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    pipeline,
    logging,
)

# load the quantized settings, we're doing 4 bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False,
)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    # use the gpu
    device_map={"": 0}
)

# don't use the cache
model.config.use_cache = False

# Load the tokenizer from the model (llama2)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True, use_fast=False)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

2024-03-06 08:22:47.377639: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see solightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-06 08:22:47.426613: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

# Testing the base model

In [3]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)


prompt = "If you're writing a library, or an app, where do the unit test files go? It's nice to separate the test files from the main app code, but it's awkward to put them into a 'tests' subdirectory inside of the app root directory, because it makes it harder to import the modules that you'll be testing. Is there a best practice here?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] If you're writing a library, or an app, where do the unit test files go? It's nice to separate the test files from the main app code, but it's awkward to put them into a 'tests' subdirectory inside of the app root directory, because it makes it harder to import the modules that you'll be testing. Is there a best practice here? [/INST] In software development, it's common to keep unit tests separate from the main application code for organization, maintainability, and testability reasons. However, you're right that putting tests in a 'tests' subdirectory within the app root directory can make importing the modules being tested more difficult.

Instead, many developers prefer to place the unit tests in a separate directory at the same level as the application code. This is often referred to as the "test suite" or "test folder." This approach makes it easier to import the modules being


# Fine tuning the model 

In [4]:
from datasets import load_dataset
from datasets import Dataset
# Load the dataset
dataset = load_dataset(dataset_name, split="train")

df = dataset.to_pandas()

# Create the new 'text' column by concatenating the formatted text
df['text'] = '<s>[INST] ' + df['question'] + ' [/INST] (' + df['answer'] + ') </s>'

# Keep only the 'text' column in the new dataset
new_df = df[['text']]
# Convert DataFrame to dataset
new_df = Dataset.from_pandas(new_df)

In [5]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)
num_train_epochs = 10
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,      
    per_device_train_batch_size=4,          
    gradient_accumulation_steps=2,          
    optim="paged_adamw_32bit",              
    save_steps=0,                           
    logging_steps=10,                       
    learning_rate=2e-3,                     
    weight_decay=0.001,                     
    fp16=False,                            
    bf16=False,                             
    max_grad_norm=0.3,                     
    max_steps=-1,                           
    warmup_ratio=0.03,                      
    group_by_length=True,                   
    lr_scheduler_type="cosine",           
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=new_df,
    peft_config=peft_config,                
    dataset_text_field="text",
    max_seq_length=100,                    
    tokenizer=tokenizer,                   
    args=training_arguments,                
    packing=False,                          
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)


/home/laughing_bhabha/.local/lib/python3.8/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/3296 [00:00<?, ? examples/s]

{'loss': 3.4308, 'grad_norm': 1.2479355335235596, 'learning_rate': 0.00016129032258064516, 'epoch': 0.02}
{'loss': 2.544, 'grad_norm': 0.7114229798316956, 'learning_rate': 0.0003225806451612903, 'epoch': 0.05}
{'loss': 2.1858, 'grad_norm': 0.6336128115653992, 'learning_rate': 0.0004838709677419355, 'epoch': 0.07}
{'loss': 2.1411, 'grad_norm': 0.5810688734054565, 'learning_rate': 0.0006451612903225806, 'epoch': 0.1}
{'loss': 1.936, 'grad_norm': 0.9818006157875061, 'learning_rate': 0.0008064516129032258, 'epoch': 0.12}
{'loss': 1.9895, 'grad_norm': 0.5644474029541016, 'learning_rate': 0.000967741935483871, 'epoch': 0.15}
{'loss': 1.8008, 'grad_norm': 0.6513486504554749, 'learning_rate': 0.0011290322580645162, 'epoch': 0.17}
{'loss': 1.8841, 'grad_norm': 0.607361376285553, 'learning_rate': 0.0012903225806451613, 'epoch': 0.19}
{'loss': 1.8131, 'grad_norm': 2.50314998626709, 'learning_rate': 0.0014516129032258066, 'epoch': 0.22}
{'loss': 1.7884, 'grad_norm': 0.9189577102661133, 'learning_r

# Testing the fine tuned model 

In [10]:
# Ignore warnings
logging.set_verbosity(logging.CRITICAL)

# Run text generation pipeline with our next model
prompt = "I have a multi-line string literal that I want to do an operation on each line, like so: inputString = '''Line 1 Line 2 Line 3''' I want to do something like the following: for line in inputString: doStuff()"

pipe = pipeline(task="text-generation", model=trainer.model, tokenizer=tokenizer, max_length=200)
result = pipe(f"[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

[INST] I have a multi-line string literal that I want to do an operation on each line, like so: inputString = '''Line 1 Line 2 Line 3''' I want to do something like the following: for line in inputString: doStuff() [/INST] (You could also use the builtin function map:
def map(a):
    return map(lambda x: x[0]+a]

) 

)  - only works for Python 2.7 or later
) 

as you can't get the whole line right away without the'map'.
for line in inputLine:
    doSomething(line)

) 

# Example 1
line_1 line_2 line_3
do-something(line_1) 

can also be used for dynamic operations on multiple lines (like assigning multiple variables values to a collection
